# 02 · Préparation des données

Notebook dédié au nettoyage, à la fusion et à l'enrichissement des jeux de données olympiques avant passage aux modèles de clustering et de prédiction.

## Plan de travail
- Charger les sources brutes (résultats, médailles, athlètes).
- Normaliser la structure des résultats (exploser la colonne des athlètes).
- Fusionner les informations individuelles et pays.
- Créer des agrégations par pays/édition pour les modèles.
- Sauvegarder les jeux nettoyés dans `data/processed`.

In [ ]:
import ast
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)
np.random.seed(42)

In [ ]:
BASE_DIR = Path('..').resolve()
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw' if (DATA_DIR / 'raw').exists() else DATA_DIR
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

RAW_DIR

In [ ]:
results_path = RAW_DIR / 'olympic_results.csv'
medals_path = RAW_DIR / 'olympic_medals.csv'
athletes_path = RAW_DIR / 'olympic_athletes.csv'

results_df = pd.read_csv(results_path)
medals_df = pd.read_csv(medals_path)
athletes_df = pd.read_csv(athletes_path)

results_df.shape, medals_df.shape, athletes_df.shape

In [ ]:
results_df.columns.tolist()
# aperçu des colonnes disponibles

## Normalisation de la colonne des athlètes
Certaines épreuves listent plusieurs athlètes sous forme de listes encodées en chaîne de caractères. L'objectif est de les exploser pour travailler au niveau d'un athlète par ligne.

In [ ]:
def parse_athlete_list(cell):
    if isinstance(cell, list):
        return cell
    if isinstance(cell, str) and cell.startswith('['):
        try:
            parsed = ast.literal_eval(cell)
            return [
                {
                    'athlete_full_name': item[0],
                    'athlete_url': item[1]
                }
                for item in parsed
                if isinstance(item, (list, tuple)) and len(item) >= 2
            ]
        except (ValueError, SyntaxError):
            return []
    return []

results_with_list = results_df.copy()

if 'athlete_records' in results_with_list.columns:
    source_col = 'athlete_records'
elif 'athletes' in results_with_list.columns:
    source_col = 'athletes'
else:
    source_col = None

if source_col is not None:
    results_with_list['athlete_records'] = results_with_list[source_col].apply(parse_athlete_list)
else:
    results_with_list['athlete_records'] = [[] for _ in range(len(results_with_list))]

results_exploded = results_with_list.explode('athlete_records', ignore_index=True)
athlete_details = results_exploded['athlete_records'].apply(pd.Series)

results_tidy = pd.concat(
    [results_exploded.drop(columns=['athlete_records']), athlete_details],
    axis=1
)

results_tidy.head(3)

## Fusion avec les médailles et les métadonnées athlètes
On consolide les informations de médaille et le profil des athlètes pour enrichir le dataset final.

In [ ]:
medals_trimmed = medals_df.rename(columns={'medal_type': 'medal_type_medals'})

full_df = results_tidy.merge(
    medals_trimmed[['athlete_url', 'slug_game', 'event_title', 'medal_type_medals']],
    on=['athlete_url', 'slug_game', 'event_title'],
    how='left'
)

full_df = full_df.merge(
    athletes_df,
    on='athlete_url',
    how='left',
    suffixes=('', '_profile')
)

full_df['medal_type_result'] = full_df.get('medal_type')
full_df['medal_type_final'] = full_df['medal_type_result'].fillna(full_df['medal_type_medals'])
full_df['medal_flag'] = full_df['medal_type_final'].notna().astype(int)

full_df.head(3)

## Agrégations pays/édition
Création des features agrégées qui serviront aux modèles de clustering et de prévision du nombre de médailles.

In [ ]:
aggregation_map = {
    'medal_flag': 'sum',
    'athlete_full_name': 'nunique',
    'rank_position': 'mean'
}

country_year_summary = (
    full_df.groupby(['country_name', 'slug_game'], dropna=False)
          .agg(aggregation_map)
          .rename(columns={
              'medal_flag': 'medals_total',
              'athlete_full_name': 'athletes_unique',
              'rank_position': 'avg_rank'
          })
          .reset_index()
)

country_year_summary.head(3)

In [ ]:
full_path = PROCESSED_DIR / 'olympic_full.csv'
summary_path = PROCESSED_DIR / 'country_year_summary.csv'

full_df.to_csv(full_path, index=False)
country_year_summary.to_csv(summary_path, index=False)

full_path, summary_path

## Étapes suivantes
- Documenter dans `docs/README_methodo.md` les choix de nettoyage et les hypothèses.
- Réinjecter les agrégations dans `03_clustering.ipynb` et `05_prediction_medals.ipynb`.
- Versionner les fichiers produits dans `data/processed` et consigner leur date de génération.